# **C-calculate_onset_date.ipynb**

Author: Zhixian Yang

Email: [yangzhx28@mail2.sysu.edu.cn](mailto:yangzhx28@mail2.sysu.edu.cn) or [yimu01439@gmail.com](mailto:yimu01439@gmail.com)

GitHub: [https://github.com/koar-create](https://github.com/koar-create)

Date created: July 27th, 2023

Last modified: July 27th, 2023

<br><br>

---

<br><br>

## **Description**
This document is a Jupyter Notebook designed for an exercise derived from the "Computational Tools for Climate Science 2023" course offered by Climatematch Academy. The code presented here comprises a combination of materials provided in the course and code obtained from online sources.

# <font color='red'>1. download and import packages</font>

In [1]:
!pip install cdsapi --quiet
!pip install imageio[ffmpeg] --quiet
!pip install imageio[pyav] --quiet

import imageio
import cdsapi
import h5py
import urllib.request
from itertools import product
import s3fs, boto3, botocore, pooch
from pythia_datasets import DATASETS
import os, sys, glob, platform, tempfile
from datetime import datetime, timedelta
import numpy as np, pandas as pd, xarray as xr
import matplotlib as mpl, matplotlib.pyplot as plt
import cartopy, cartopy.crs as ccrs, cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.io.shapereader as shapereader

# <font color='red'>2. read data</font>

<br>

### **<font face='Arial' size=4>Yang: option 1 is the file `sst.mnmean.nc` under the folder `shared/Data/Projects/ENSO` with time resolution of 1-month.</font>**

In [2]:
# data source 1
ds = xr.open_dataset('shared/Data/Projects/ENSO/sst.mnmean.nc')
ds

# <font color='red'>3. calculate SST anomalies EOF</font>

In [ ]:
var = ds.sst.sel(lat=slice(30, -30), lon=slice(60, 240)).squeeze()

n, N, P = var.shape
var_2D = var.reshape(n, N * P).transpose()  # reshape and transpose it, making it has the shape of (NP, n)

# test: does using the reshape() function disrupt the order of indices?
print('reshape() function will not disrupt the order of indices: ', var_2D.transpose.reshape(n, N, P) == var)

X = var_2D @ var_2D.transpose()

eigvals, eigvecs = np.linalg.eig(X)

In [ ]:
sorted_indices = np.argsort(eigvals)[::-1]

# order eigenvalues in descending order
sorted_eigenvals = eigvals[sorted_indices]

# order eigenvectors in descending order, that is EOF. 
EOF = eigvecs[:, sorted_indices]
PC = EOF.transpose() @ var_2D

EOF1 = EOF[:, 0].reshape(N, P)
PC1 = PC[0, :]

In [ ]:
# plot the 1st mode of EOF
proj = ccrs.PlateCarree()
fig, ax = plt.subplots(dpi=300, subplot_kw={'projection': proj})

ax.coastlines()
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='k', alpha=0.8, linewidth=1, linestyle='dashed', draw_labels=True, x_inline=False, y_inline=False)
gl.xformatter, gl.yformatter = LongitudeFormatter(), LatitudeFormatter()
gl.top_labels, gl.right_labels = False, False
gl.xlabel_style, gl.xlabel_style = {'family':'Monospace', 'size':14}, {'family':'Monospace', 'size':14}

im = ax.contourf(var.lon, var.lat, EOF1, transform=ccrs.PlateCarree(), 
                 levels=np.linspace(vmin, vmax, 11), extend='both', cmap='coolwarm')
cbar = fig.colorbar(im, ticks=np.linspace(vmin, vmax, 11), orientation='horizontal')
cbar.set_label('units: K', fontdict={'family':'Arial', 'size':16, 'weight':'bold'})

ax.set_title('EOF1', fontdict={'family':'Arial', 'size':18})

In [ ]:
# plot the 1st principal component of sst anomalies
fig, ax = plt.subplots(figsize=(9, 6))

ax.plot(var.time, PC1, linewidth=1, linestyle='-', color='darkblue')

ax.grid(True, linestyle='dashed', linewidth=0.8, color='k', alpha=0.8)
ax.axhline(0.0, linewidth=2, linestyle='-', color='F00000')

ax.set_title('PC1', fontdict={'family':'Arial', 'size':18})
ax.set_xlabel('time', fontdict={'family':'Arial', 'size':16})
ax.set_ylabel('PC1', fontdict={'family':'Arial', 'size':16})